In [4]:
import requests
from PIL import Image
import io
import tensorflow as tf
import torch
import datetime
from pathlib import Path
from datasets import DatasetDict
import time

In [5]:
from datasets import load_dataset

dataset = load_dataset("stochastic/random_streetview_images_pano_v0.0.2")

Using custom data configuration stochastic--random_streetview_images_pano_v0.0.2-65d25597b51c2d04
Found cached dataset parquet (/Users/derekyu/.cache/huggingface/datasets/stochastic___parquet/stochastic--random_streetview_images_pano_v0.0.2-65d25597b51c2d04/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
# preprocessing - this took 6min 48 seconds on my computer... jk 12 min second time
from torchvision.transforms import v2
import torchvision.transforms as transforms
# data pre processing
transform = v2.Compose([
    v2.CenterCrop((561,1010)), # crops the middle image from the panorama
    v2.Resize((224,224)), # resizing the image to 224x224 for easier processing
    transforms.ToTensor() # converting the image to a tensor
])

# Create a new list to hold the transformed dataset
transformed_dataset = {'train': []}

# Loop through the original dataset and apply the transformation
for item in dataset['train']:
    transformed_item = item.copy()  # Make a copy of the current item
    transformed_item['image'] = transform(item['image'])  # Transform the image
    transformed_dataset['train'].append(transformed_item)  # Add to the new dataset

# for i in range(len(dataset['train'])):
#     dataset['train'][i]['image'] = transform(dataset['train'][i]['image'])

In [24]:
transformed_dataset['train'][0]
transformed_dataset['train'][0]['image']

tensor([[[0.5647, 0.6000, 0.6667,  ..., 0.5843, 0.5922, 0.5922],
         [0.5804, 0.5255, 0.5255,  ..., 0.5843, 0.5922, 0.5922],
         [0.5176, 0.5451, 0.5412,  ..., 0.5843, 0.5922, 0.5922],
         ...,
         [0.5961, 0.5843, 0.5922,  ..., 0.5765, 0.5725, 0.5569],
         [0.5882, 0.6196, 0.6157,  ..., 0.5882, 0.5725, 0.5686],
         [0.5804, 0.6431, 0.6118,  ..., 0.5725, 0.5569, 0.5255]],

        [[0.5373, 0.5804, 0.6863,  ..., 0.7451, 0.7490, 0.7490],
         [0.5529, 0.5020, 0.5373,  ..., 0.7451, 0.7490, 0.7490],
         [0.4863, 0.5098, 0.5373,  ..., 0.7451, 0.7490, 0.7490],
         ...,
         [0.5608, 0.5490, 0.5569,  ..., 0.5451, 0.5490, 0.5294],
         [0.5529, 0.5843, 0.5804,  ..., 0.5529, 0.5490, 0.5412],
         [0.5490, 0.6078, 0.5725,  ..., 0.5333, 0.5294, 0.4980]],

        [[0.5373, 0.6353, 0.6941,  ..., 0.9882, 0.9922, 0.9922],
         [0.5098, 0.5176, 0.5137,  ..., 0.9882, 0.9922, 0.9922],
         [0.4275, 0.5098, 0.4980,  ..., 0.9882, 0.9922, 0.

In [13]:
# dataset
train_testvalid = dataset['train'].train_test_split(test_size=0.4)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']
})
# datasets.save_to_disk("./data")

DatasetDict({
    train: Dataset({
        features: ['image', 'country_iso_alpha2', 'latitude', 'longitude', 'address'],
        num_rows: 6632
    })
    test: Dataset({
        features: ['image', 'country_iso_alpha2', 'latitude', 'longitude', 'address'],
        num_rows: 2211
    })
    valid: Dataset({
        features: ['image', 'country_iso_alpha2', 'latitude', 'longitude', 'address'],
        num_rows: 2211
    })
})

In [1]:
country_codes = ["ZA","KR","AR","BW","GR","SK","HK","NL","PE","AU","KH","LT","NZ","RO","MY","SG","AE","FR","ES","IT","IE","LV","IL","JP","CH","AD","CA","RU","NO","SE","PL","TW","CO","BD","HU","CL","IS","BG","GB","US","SI","BT","FI","BE","EE","SZ","UA","CZ","BR","DK","ID","MX","DE","HR","PT","TH"]
country_dict = {}
# TODO: these might need to be tensor arrays but thats easy enough to change if needed
for i in range(len(country_codes)):
    country_dict[country_codes[i]] = [0]*len(country_codes)
    country_dict[country_codes[i]][i] = 1
# print(country_dict)


{'ZA': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'KR': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'AR': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'BW': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'GR': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'SK': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [11]:
# print(datasets)
# datasets['train'][0]['image']
type(datasets['train'][0]['image'])

PIL.JpegImagePlugin.JpegImageFile

In [11]:
# referenced: https://blog.paperspace.com/convolutional-autoencoder/
# autoencoder classes (CREDIT: LARGELY TAKEN FROM 6_AUTOENCODER NOTEBOOK, but encoder and decoder architectures modified to be convolutional)
# should only have Encoder that has a latent dimension of 50 - corresponding to country weights

class MLPEncoder(torch.nn.Module):

    def __init__(self,
                 number_of_hidden_layers: int,
                 latent_size: int,
                 hidden_size: int,
                 input_size: int,
                 activation: torch.nn.Module):

        super().__init__()

        self.latent_size = latent_size
        assert number_of_hidden_layers >= 0, "Decoder number_of_hidden_layers must be at least 0"

        layers = []
        in_channels = 3
        out_channels = 16
        layers.append(torch.nn.Conv2d(in_channels, #in_channels
                                      out_channels, #out_channels
                                      3, #kernel_size
                                      stride=2, #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        # 32 x 32
        layers.append(torch.nn.Conv2d(out_channels, #in_channels
                                      out_channels, #out_channels
                                      3, #kernel_size
                                      stride=1, #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        layers.append(torch.nn.Conv2d(out_channels, #in_channels
                                      out_channels * 2, #out_channels
                                      3, #kernel_size
                                      stride=2,  #stride
                                      padding=1 
                                      ))
        layers.append(activation)
        #16 x 16
        layers.append(torch.nn.Conv2d(out_channels * 2, #in_channels
                                      out_channels * 2, #out_channels
                                      3, #kernel_size
                                      stride=1,  #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        # 8x8 feature maps
        # 64 out channels
        layers.append(torch.nn.Conv2d(out_channels * 2, #in_channels
                                      out_channels*4, #out_channels
                                      3, #kernel_size
                                      stride=2,  #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        # input is now 8x8 feature maps for out_channels*4 channels.
        layers.append(torch.nn.Flatten())
        #flatten to latent_size 
        layers.append(torch.nn.Linear(4*out_channels*8*8 , # features in
                                      latent_size # features out
                                      ))
        layers.append(activation)
        self.net = torch.nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        #return torch.rand(1,self.latent_size)
        return self.net(x)


Loss = torch.nn.BCELoss()

In [16]:
# define our training parameters and model
hidden_layers = 4
hidden_size = 30

latent_size = 55
## this might need to change
input_size = 64
lr = 0.001
# lambda weight for classifier's loss
lamb = 1

# fix random seed
torch.manual_seed(0)

# select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPEncoder( number_of_hidden_layers=hidden_layers,
                 latent_size=latent_size,
                 hidden_size=hidden_size,
                 input_size=input_size,
                 activation=torch.nn.ReLU()).to(device)

# use an optimizer to handle parameter updates
opt = torch.optim.Adam(model.parameters(), lr=lr)

# save all log data to a local directory
run_dir = "logs"

# to clear out TensorBoard and start totally fresh, we'll need to
# remove old logs by deleting them from the directory
!rm -rf ./logs/

# timestamp the logs for each run so we can sort through them
run_time = datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")

# initialize a SummaryWriter object to handle all logging actions
logger = SummaryWriter(log_dir=Path(run_dir) / run_time, flush_secs=20)

In [22]:
# load data from disk
data = datasets.load_from_disk("./data")
data = data.with_format("torch")

In [ ]:
# (***credit***: mostly taken from provided notebook )
# training

epochs = 100
start_time = time.time()
loss_history = []
valid_history = []
acc_history = []
valid_acc_history = []

# split this into batches
for epoch in range(epochs):

    # weight batch losses/scores proportional to batch size
    iter_count = 0
    valid_iter_count = 0
    loss_epoch = 0
    class_accuracy_epoch = 0
    valid_loss_epoch = 0
    valid_accuracy_epoch = 0
    ###
    ### IMAGE_DATA_TRAIN is training data, shape is 610 x 3 x 64 x 64
    ### 
    
   # print(batched_image_data_train[0][0].shape)

    # test with literally only one image
    #batched_image_data_train = [batched_image_data_train[0][0].unsqueeze(0)]
    for idx, img in enumerate(datasets['train']):
        
        x = img['image']
        # print(x)
        # flatten input images and move to device\
        # *****
        x = x / 255
        # plot x_real later to see if this is correct
        x = x.to(device)
        model.zero_grad()

        # train on a batch of inputs
        pred_labels = model(x_real)

        # get the true label
        label = country_dict[img['country_code']]
        loss = Loss(label, pred_labels)
        loss.backward()
        opt.step()

        # log loss
        loss_epoch += loss.detach().item()
        iter_count += 1

        # apply the .5 threshold to the classification
        true_class = int(true_label_tensor[0])
        c = 1 if (classification.mean() > 0.5) else 0   
    

        # classification accuracy
        acc = (true_class == c) 
        class_accuracy_epoch += acc

        # print(f"true: {true_class}")
        # print(f"pred: {classification.mean()}")
        # print(f"acc: {acc}")

    # plot loss
    loss_epoch /= iter_count
    class_accuracy_epoch /= iter_count
   # print(iter_count)
    logger.add_scalar("mse_loss", loss_epoch, epoch)
    loss_history.append(loss_epoch)
    acc_history.append(class_accuracy_epoch)
 
            
            # logger.add_scalar("mse_loss_valid", valid_loss_epoch, epoch)
    # # plot example generated images
    # with torch.no_grad():
    #     reconstructed_batch = model(example_batch.reshape(batch_size, -1)).reshape(batch_size, 1, image_size, image_size)
    #     logger.add_image("reconstructed_images", make_grid(reconstructed_batch, math.floor(math.sqrt(batch_size)), title="Reconstructed Images"), epoch)
        # calculate validation loss

    with torch.no_grad():
        for batch_idx, batch_data in enumerate(batched_image_data_valid):
            x_real = batch_data.float()
            x_real = x_real / 255
            x_real = x_real.to(device)
            x_reconstructed, classification = model(x_real)

            true_label_tensor = torch.tensor(labels['valid'][batch_idx]).repeat(5,1).float()
            valid_loss = loss([x_real,classification], [x_reconstructed, true_label_tensor])
            valid_loss_epoch += valid_loss.detach().item() * valid_batch_size

            
                # apply the .5 threshold to the classification
            true_class = int(true_label_tensor[0])
            c = 1 if (classification.mean() > 0.5) else 0   

            

            # classification accuracy
            valid_acc = (true_class == c) * valid_batch_size
            valid_accuracy_epoch += valid_acc
            valid_iter_count += valid_batch_size
            # print(f"true: {true_class}")
            # print(f"pred: {classification.mean()}")
            # print(f"acc: {valid_acc}")
        valid_loss_epoch /= valid_iter_count
        valid_history.append(valid_loss_epoch)
        valid_accuracy_epoch /= valid_iter_count
        valid_acc_history.append(valid_accuracy_epoch)

    if (epoch + 1) % report_every == 0:
        mins = (time.time() - start_time) / 60
        print(f"Epoch: {epoch + 1:5d}\tMSE Loss: {loss_epoch :6.4f}\t in {mins:5.1f}min")
        print()